# NLP with Disasters Tweets

## Import Stuff yada yada

In [1]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import torch
from transformers import BertTokenizer, BertModel

from sklearn import feature_extraction, model_selection, metrics
from sklearn import naive_bayes, ensemble, tree, linear_model
import xgboost

import warnings
warnings.filterwarnings('ignore')
plt.style.use('dark_background')

DUMMY_RUN = True

In [2]:
train_df = pd.read_csv('train.csv')

## EDA

In [3]:
tweet_len = train_df[train_df['target']==1]['text'].str.len()

## Preprocessing

In [4]:
stemmer = nltk.stem.PorterStemmer()

def clean(text):
    new_text = remove_links(text.lower())
    new_text = re.sub('[^a-zA-Z ]', '', new_text)
    new_text = stemmer.stem(new_text)
    return new_text

def remove_links(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

In [5]:
tweets = [clean(tweet) for tweet in train_df['text']]

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

def bert_tokenize(text):
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  return output

# X = [bert_tokenize(tweet) for tweet in tweets]
y = train_df['target']


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
print(bert_tokenize("i hate everything"))

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.1915,  0.4895, -0.2911,  ..., -0.0122,  0.1492,  0.1083],
         [ 0.1957,  0.3231, -0.6850,  ...,  0.2040,  0.8128,  0.1034],
         [ 0.5748,  0.5154, -0.1396,  ...,  0.1337,  0.3736,  0.1703],
         [-0.1849,  0.2354, -0.4561,  ...,  0.9887,  0.8068, -0.1834],
         [ 0.8280,  0.0850, -0.2614,  ..., -0.2030, -0.5610, -0.4148]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-7.8596e-01, -2.3175e-01,  4.5795e-01,  5.5659e-01, -1.9156e-01,
         -1.2896e-01,  7.4936e-01,  1.5217e-01,  7.7843e-02, -9.9923e-01,
          2.7356e-01,  3.2724e-01,  9.6944e-01, -2.2192e-01,  8.6324e-01,
         -4.6482e-01, -1.1516e-01, -4.8018e-01,  2.6196e-01, -6.2988e-01,
          4.5500e-01,  7.5136e-01,  5.8449e-01,  2.0188e-01,  3.3682e-01,
          4.1782e-01, -4.5399e-01,  8.5607e-01,  9.1431e-01,  6.0741e-01,
         -5.6835e-01,  9.5383e-02, -9.7337e-01, -7.3753e-02,  4.0617e-01,
    

## Model Selection

In [ ]:
dict_classifiers = {
    'xgb': xgboost.XGBClassifier(),
    'gaussian_nb': naive_bayes.GaussianNB(), # why nan?
    'multi_nb': naive_bayes.MultinomialNB(),
    # 'rforest': ensemble.RandomForestClassifier(),
    'decision_tree': tree.DecisionTreeClassifier(),
    'log_reg': linear_model.LogisticRegression()
}

In [ ]:
if DUMMY_RUN:
    for name in dict_classifiers:
        clf = dict_classifiers[name]
        scores = model_selection.cross_val_score(clf, X, y)
        print(f"{name} score: {np.average(scores)}")

xgb score: 0.5582559320381396
gaussian_nb score: 0.5485410305236484
multi_nb score: 0.5227933836236834
decision_tree score: 0.5585185715653885
log_reg score: 0.5606208957181302


## Fitting & Forecast

In [ ]:
choice = 'multi_nb'
clf = dict_classifiers[choice]

In [ ]:
if not DUMMY_RUN:
    # Importing and pre-processing test data
    test_df = pd.read_csv('test.csv')
    test_tweets = [clean(tweet) for tweet in test_df['text']]
    X_test = cv.transform(test_tweets)
    
    # Fit & forecast
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    # Export
    out = pd.DataFrame(
        data={'id': test_df['id'],
              'target': y_pred},
    )
    out.to_csv('results.csv', index = False)